# Workflow:
1. Randomly Sample 200 articles for each outlet, and concatenate to a dataframe
2. Label each outlet with the associated outlet_labels and merge
3. Get the average bias score from reference_df for each outlet and merge with data
4. Compare outlet_labels and bias scores in order to determine left/right bias
5. Start preprocessing and model building
6. Account for class-imbalance and feature selection

### Notes:
- outlet_labels are based on mediabiasfactcheck.com
- reference_df is based on adfontesmedia.com
- df is based on kaggle.com/snapcrack/all-the-news

# Data importing and creation

In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("data/articles1.csv")
df2 = pd.read_csv("data/articles2.csv")
df3 = pd.read_csv("data/articles3.csv")
df = pd.concat([df1, df2, df3])

reference_df = pd.read_csv("data/Interactive Media Bias Chart - Ad Fontes Media.csv")

In [3]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [4]:
# Getting unique outlets
outlets = sorted(df['publication'].unique())

In [5]:
outlets

['Atlantic',
 'Breitbart',
 'Business Insider',
 'Buzzfeed News',
 'CNN',
 'Fox News',
 'Guardian',
 'NPR',
 'National Review',
 'New York Post',
 'New York Times',
 'Reuters',
 'Talking Points Memo',
 'Vox',
 'Washington Post']

In [6]:
# Extreme Left, Left, Left-Center, Center, Right-Center, Right, Extreme Right
outlet_labels = ['Left-Center', 'Extreme Right', 'Left-Center', 'Left-Center', 'Left', 'Right', 'Left-Center', 'Left-Center',
               'Right', 'Right-Center', 'Left-Center', 'Center', 'Left', 'Left', 'Left-Center']

outlet_df = pd.DataFrame({'outlet': outlets, 
                          'label':outlet_labels})

In [7]:
outlet_df.head()

,outlet,label
0,Atlantic,Left-Center
1,Breitbart,Extreme Right
2,Business Insider,Left-Center
3,Buzzfeed News,Left-Center
4,CNN,Left


In [8]:
# A negative bias is Left Leaning, a positive bias is Right Leaning
reference_df.head()

,Source,Url,Bias,Quality
0,ABC,https://abcnews.go.com/Politics/us-disrupted-a...,-5.33,52.33
1,ABC,https://abcnews.go.com/Politics/appeals-court-...,0.67,51.67
2,ABC,https://abcnews.go.com/Politics/electoral-coll...,-10.00,32.00
3,ABC,https://abcnews.go.com/Politics/facebook-agree...,-2.33,52.33
4,ABC,https://abcnews.go.com/Politics/donald-trump-t...,-4.33,52.67


In [9]:
reference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 4 columns):
Source     1916 non-null object
Url        1916 non-null object
Bias       1916 non-null float64
Quality    1916 non-null float64
dtypes: float64(2), object(2)
memory usage: 60.0+ KB


# Preprocessing
##### We have 3 dataframes:
1. df -> main data, with articles and features
2. outlet_df -> political labels for each outlet
3. reference_df -> political bias scores for each article from each outlet

### Processing reference_df to get the average bias score per outlet

In [10]:
# Renaming entities in reference_df to match data
reference_df['Source'].replace({'The Atlantic': 'Atlantic',
                                'BuzzFeed': 'Buzzfeed News',
                                'The Guardian': 'Guardian', 
                                'National Public Radio': 'NPR'}, inplace=True)

In [11]:
# Filtering to outlets present in data
reference_df = reference_df[reference_df['Source'].isin(outlets)]

In [12]:
# Getting mean bias score for each outlet
reference_df = reference_df[['Source', 'Bias']].groupby('Source').mean()

In [13]:
# Adding index and renaming
reference_df.reset_index(level=0, inplace=True)
reference_df.rename(columns={"Source": "outlet"}, inplace=True)

In [14]:
reference_df.head()

,outlet,Bias
0,Atlantic,-6.410714
1,Breitbart,18.987857
2,Business Insider,-0.378000
3,Buzzfeed News,-7.061333
4,CNN,-8.553827


### Sampling 200 articles from each outlet and concatenating to dataframe

In [15]:
df_list = []
for outlet in outlets:
    df_list.append(df[df['publication'] == outlet].sample(200))

df_sample = pd.concat(df_list)

In [16]:
# Dropping unused features
df_sample.drop(columns=['Unnamed: 0', 'id', 'date', 'month', 'url'], inplace=True)

In [17]:
df_sample.head()

,title,publication,author,year,content
4346,Congress Falls Short on Zika,Atlantic,Nora Kelly,2016.0,", I want to receive updates from partners and ..."
3522,Quoted: The Under-the-Radar Edition,Atlantic,Rosa Inocencio Smith,2016.0,When you don’t know what you don’t know: “[...
5033,The Atlantic Politics & Policy Daily: RNC Goi...,Atlantic,Elaine Godfrey,2016.0,For us to continue writing great stori...
2384,David Cameron and the Panama Papers,Atlantic,NaN,2016.0,", I want to receive updates from partners and ..."
1473,An Uncomfortably Familiar Problem for a Trump ...,Atlantic,Matt Ford,2017.0,BuzzFeed first published excerpts of Gorsuch’s...


### Merging the 3 dataframes

In [18]:
final_reference_df = reference_df.merge(outlet_df, on='outlet')

In [19]:
final_reference_df.head()

,outlet,Bias,label
0,Atlantic,-6.410714,Left-Center
1,Breitbart,18.987857,Extreme Right
2,Business Insider,-0.378000,Left-Center
3,Buzzfeed News,-7.061333,Left-Center
4,CNN,-8.553827,Left


In [20]:
df_sample.rename(columns={"publication": "outlet"}, inplace=True)
df_sample = df_sample.merge(final_reference_df, on='outlet')

In [21]:
df_sample.head(10)

,title,outlet,author,year,content,Bias,label
0,Congress Falls Short on Zika,Atlantic,Nora Kelly,2016.0,", I want to receive updates from partners and ...",-6.410714,Left-Center
1,Quoted: The Under-the-Radar Edition,Atlantic,Rosa Inocencio Smith,2016.0,When you don’t know what you don’t know: “[...,-6.410714,Left-Center
2,The Atlantic Politics & Policy Daily: RNC Goi...,Atlantic,Elaine Godfrey,2016.0,For us to continue writing great stori...,-6.410714,Left-Center
3,David Cameron and the Panama Papers,Atlantic,NaN,2016.0,", I want to receive updates from partners and ...",-6.410714,Left-Center
4,An Uncomfortably Familiar Problem for a Trump ...,Atlantic,Matt Ford,2017.0,BuzzFeed first published excerpts of Gorsuch’s...,-6.410714,Left-Center
5,Is Target Losing Its Cool?,Atlantic,Bourree Lam,2017.0,One reason Target’s store sales are declining ...,-6.410714,Left-Center
6,A Trip to a Swedish Town That’s Being Complete...,Atlantic,Tom Bradstreet,2016.0,For us to continue writing great stori...,-6.410714,Left-Center
7,How Blacks and Whites Spend Differently,Atlantic,Gillian B. White,2016.0,", I want to receive updates from partners and ...",-6.410714,Left-Center
8,Why Istanbul Banned LGBT Rights Marches,Atlantic,Matt Vasilogambros,2016.0,", I want to receive updates from partners and ...",-6.410714,Left-Center
9,What Will Make the Killings Stop?,Atlantic,Vann R. Newkirk II,2016.0,I want to receive updates from partners and...,-6.410714,Left-Center
